# Planificación del proyecto <a name="Planificacion"></a>

* [Planificación del proyecto](#Planificacion)
* [Entenidimiento del negocio](#Entenidimiento)
* [Comprensión de los datos](#Comprension)
    * [Importamos librerias](#Importamos)
    * [Cargamos el dataset](#Cargamos)
* [Preparación de los datos](#Preparacion)
* [Modelado Machinel Learning](#ModeladoML)
    * [Linear Regression](#LinearR)
        * [Ridge Regression](#Ridge)
* [Modelado Machinel Learning](#ModeladoDL)
* [Evaluación](#Evaluacion)
* [Subir el modelo a Kaggle](#Subir)

## Entenidimiento del negocio <a name="Entenidimiento"></a>

En esta práctica se deben estimar los costes médidos de diferentes pacientes usando un modelo de Machine Learning, dicha practica se podrá constrastar con las compañeras en la competición de Kaggle.
https://www.kaggle.com/c/estimacin-de-costes-mdicos-sic-ed2-2021/overview

## Comprensión de los datos <a name="Comprension"></a>

Nos han dado dos ficheros csv divididos en test y train.
Las columnas que contienen son las siguientes:
- id: columna identificativa para PK
- age: edad del principal beneficiario del seguro médico.
- sex: sexo del tomador del seguro médico.
- bmi: indice de masa corporal.
- children: número de hijos cubiertos por el seguro médico / número de descendientes.
- smoker: fumador.
- region: área residencial del beneficiario del seguro médico.
- charges: costes médicos cargados a la aseguradora. Dicha columna no la tiene test.

A continuación cargaremos los datos para su correspondiente procesamiento.

### Importamos librerias <a name="Importamos"></a>

In [84]:
import pandas as pd
import numpy as np

from sklearn import datasets,decomposition
from sklearn import metrics, preprocessing
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import adam_v2


### Cargamos el dataset <a name="Cargamos"></a>

En este caso tenemos el dataset dividido en dos ficheros, la parte de entrenamiento en _train.csv_ y la parte de validación en _test.csv_, por lo tanto creamos dos variables para ellos y hacemos drop de la PK ya que no es necesaria.

In [21]:
#Parte para el entrenamiento
dataset_train = pd.read_csv('train.csv')
#dataset_train = dataset_train.drop('id', 1) # Para el entrenamiento no usamos los ids

#Parte para la validacion
dataset_test = pd.read_csv('test.csv')
#dataset_test = dataset_test.drop('id', 1) # Para la validacion no usamos los ids

dataset_train.head()
#dataset_test.head()

,id,age,sex,bmi,children,smoker,region,charges
0,1229,58,male,30.305,0,no,northeast,11938.25595
1,1073,54,female,28.880,2,no,northeast,12096.65120
2,768,64,female,39.700,0,no,southwest,14319.03100
3,606,27,female,25.175,0,no,northeast,3558.62025
4,342,60,female,27.550,0,no,northeast,13217.09450


Podemos comprobar que tenemos tres variables que no son númericas que son ``sex``, ``smoker`` y ``region``. De las cuales ``sex`` y ``smoker`` son variables binarias, a continuación comprobamos los datos que puede tener ``region`` para saber como prepararla en el siguiente paso.

In [22]:
dataset_train['region'].value_counts()

southeast    251
northeast    235
northwest    231
southwest    219
Name: region, dtype: int64

In [23]:
dataset_train['sex'].value_counts()

male      481
female    455
Name: sex, dtype: int64

In [24]:
dataset_train['smoker'].value_counts()

no     733
yes    203
Name: smoker, dtype: int64

Comprobamos que en este caso, region tiene cuatro posibilidades, será tratada en la siguiente fase.

## Preparación de los datos <a name="Preparacion"></a>

Para tener todas nuestras variables de forma numerica, pasaremos las mencionadas anteriormente por un **LabelEncoder**.

In [25]:
le = preprocessing.LabelEncoder()
# Para la parte de entrenamiento
dataset_train['sex'] = le.fit_transform(dataset_train['sex'])
dataset_train['smoker'] = le.fit_transform(dataset_train['smoker'])
dataset_train['region'] = le.fit_transform(dataset_train['region'])

# Para la parte de validación
dataset_test['sex'] = le.fit_transform(dataset_test['sex'])
dataset_test['smoker'] = le.fit_transform(dataset_test['smoker'])
dataset_test['region'] = le.fit_transform(dataset_test['region'])

dataset_train.head()

,id,age,sex,bmi,children,smoker,region,charges
0,1229,58,1,30.305,0,0,0,11938.25595
1,1073,54,0,28.880,2,0,0,12096.65120
2,768,64,0,39.700,0,0,3,14319.03100
3,606,27,0,25.175,0,0,0,3558.62025
4,342,60,0,27.550,0,0,0,13217.09450


In [26]:
type(dataset_train)

pandas.core.frame.DataFrame

Como vemos a continuación, ha dejado de tener la cadena de texto y ha pasado a tener un número categorico. 

In [27]:
dataset_train['region'].value_counts()

2    251
0    235
1    231
3    219
Name: region, dtype: int64

In [28]:
dataset_train['sex'].value_counts()

1    481
0    455
Name: sex, dtype: int64

In [29]:
dataset_train['smoker'].value_counts()

0    733
1    203
Name: smoker, dtype: int64

A continuación realizaremos un **MinMaxScaler** para normalizar las variables y que ninguna categoría de una columna tenga más peso que otra, para así entrenar a nuestro modelo correctamente.

In [30]:
dataset_train.iloc[:,1:-1]

,age,sex,bmi,children,smoker,region
0,58,1,30.305,0,0,0
1,54,0,28.880,2,0,0
2,64,0,39.700,0,0,3
3,27,0,25.175,0,0,0
4,60,0,27.550,0,0,0
...,...,...,...,...,...,...
931,60,0,32.450,0,1,2
932,62,0,39.160,0,0,2
933,55,0,29.830,0,0,0
934,20,0,33.300,0,0,3


In [31]:
# Parte para el entrenamiento
sc_train = MinMaxScaler()
sc_train.fit(dataset_train.iloc[:,1:-1]) # Se ajusta el reescalador, y omitimos la columna de id, para que no la normalice
dataset_train[['age','sex','bmi','children','smoker','region']] = sc_train.transform(dataset_train[['age','sex','bmi','children','smoker','region']])

#Parte para validacion
sc_test = MinMaxScaler()
sc_test.fit(dataset_test.iloc[:,1:]) # Se ajusta el reescalador
dataset_test[['age','sex','bmi','children','smoker','region']] = sc_test.transform(dataset_test[['age','sex','bmi','children','smoker','region']])

In [32]:
dataset_train

,id,age,sex,bmi,children,smoker,region,charges
0,1229,0.869565,1.0,0.377184,0.0,0.0,0.000000,11938.25595
1,1073,0.782609,0.0,0.337341,0.4,0.0,0.000000,12096.65120
2,768,1.000000,0.0,0.639871,0.0,0.0,1.000000,14319.03100
3,606,0.195652,0.0,0.233748,0.0,0.0,0.000000,3558.62025
4,342,0.913043,0.0,0.300154,0.0,0.0,0.000000,13217.09450
...,...,...,...,...,...,...,...,...
931,845,0.913043,0.0,0.437159,0.0,1.0,0.666667,45008.95550
932,928,0.956522,0.0,0.624773,0.0,0.0,0.666667,13470.80440
933,1091,0.804348,0.0,0.363903,0.0,0.0,0.000000,11286.53870
934,1268,0.043478,0.0,0.460925,0.0,0.0,1.000000,1880.48700


In [33]:
#Otra forma de realizar el MinMaxScaler, esta variable no se utilizara en este ejercicio
scaler = preprocessing.MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(dataset_train.iloc[:,1:]), columns=['age','sex','bmi','children','smoker','region','charges'], index=dataset_train.id)

Teniendo el dataset cargado y las variables normalizadas, ahora tendremos que separar la columna a predecir, que en este caso es ``charges`` y dividir nuestro dataset en las variables correspondientes. Tales como:
- X_train: Será nuestra fuente de datos para el entrenamiento.
- Y_train: Será nuestra fuente de datos de entrenamiento para la variable a predecir.
- X_test: Será nuestra fuente de datos de validación, la cual no le ofreceremos la variable a predecir.
- Y_test: Será nuestra fuente de datos de validación, sirve para obtener el rendimiento de nuestro modelo.

In [91]:
#MinMaxScaler nos devuelve un numpy array, lo pasamos a pandas DataFrame para poder trabajar mejor
dataset_train = pd.DataFrame(dataset_train)
dataset_test = pd.DataFrame(dataset_test)

# Se separa la columna a predecir
X_train = dataset_train.iloc[:,1:-1] #Ignoramos la columna de Id 
Y_train = dataset_train.iloc[:,-1]
X_test = dataset_test

In [77]:
Y_train

0      11938.25595
1      12096.65120
2      14319.03100
3       3558.62025
4      13217.09450
          ...     
931    45008.95550
932    13470.80440
933    11286.53870
934     1880.48700
935    48675.51770
Name: charges, Length: 936, dtype: float64

## Modelado Machinel Learning <a name="ModeladoML"></a>

Tenemos ya los datos correctos y normalizados, ahora toca elegir el tipo de modelo que vamos a desarrollar y la estructura que más se adecue, probaremos varias para constrastar datos.

En este caso, al tener claramente una variable de salida no categorica, por lo tanto descaramos _Clasificacion_ , como tenemos variable de salida, descartamos _Clustering_, nos quedaría la opción más sensata que es **Regresion**.



### Linear Regression <a name="LinearR"></a>

Realizaremos este primer modelo, asumiendo que existe una correlación lineal entre los datos. 

In [16]:
reg = LinearRegression().fit(X_train, Y_train)
print("Con los datos de entrenamiento: ", reg.score(X_train, Y_train)) #Normalmente lo comprobaria con el score de los
#datos de test, para comprobar el overfitting pero acutalmente no dispongo de Y_test

Con los datos de entrenamiento:  0.7496636971119938


In [17]:
reg.intercept_

-2145.1746072827045

Como norma general:

- Los valores positivos en estos coeficientes significan una correlación directa entre los valores de las características que representan los coeficientes y el coste medico a la aseguradora, mientras que los valores negativos representan una correlación inversa.
- La magnitud de los coeficientes miden el grado de aportación, positiva o negativa, de una determinada característica.


In [18]:
reg.coef_

array([12435.62723763,    26.01948286, 12478.8115806 ,  1517.91753992,
       23754.01188298,  -624.48267979])

In [19]:
pd.DataFrame(data=np.expand_dims(reg.coef_, axis=0), columns=['age','sex','bmi','children','smoker','region'])

,age,sex,bmi,children,smoker,region
0,12435.627238,26.019483,12478.811581,1517.91754,23754.011883,-624.48268


Como vemos los datos de **intercept** y los **coeficientes** son bastante exagerados, esto declara un overfitting que es el principal problema de este modelo, se podría solucionar con _Ridge Regression_ hagamos una prueba con el antes de pasar al siguiente modelo.

#### Ridge Regression <a name="Ridge"></a>

##### Prueba 1 de encontrar el mejor alpha

Para mejorar el modelo anterior y evitar que los outlayers afecten a nuestro resultado, vamos a quitarlos con **StandardScaler**.

También aplicaremos **PCA** su objetivo es encontrar las columnas principales y reducir la dimensionalidad del conjunto de datos .

In [20]:
std_slc = StandardScaler()
pca = decomposition.PCA() #Decomposition reduce el numero de columnas, es uno de los PCA que he entendido, 
#Es probable que exista un PCA mejor para este dataset
ridge = Ridge()

In [21]:
pipe = Pipeline(steps=[("std_slc", std_slc),
                       ("pca", pca),
                       ("ridge", ridge)])

In [22]:
n_components = list(range(1,X_train.shape[1]+1,1))
n_components

[1, 2, 3, 4, 5, 6]

Como en casi todos los modelos, existen distintas arquitecturas para el mismo modelo, cada una con sus ventajas y desventajas, añadiremos todas para que nos ofrezca la que mejor resultado de. 

Puedes mirar el funcionamiento de cada una aquí https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html?highlight=ridge#sklearn.linear_model.Ridge

Ha raíz de esto, he decidido quitar _lbfgs_ ya que aunque los datos estan normalizados, no están normalizados por el método, por lo tanto, lo he desechado para las pruebas.

In [23]:
#Creamos una lista con todos los parametros de Ridge Regression
solver = ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]

In [24]:
parameters = dict(pca__n_components=n_components,
                  ridge__normalize=[True, False],
                  ridge__alpha=np.arange(1e-3,5,0.05),
                  ridge__solver=solver)

In [25]:
clf_GS = GridSearchCV(pipe, parameters)
clf_GS.fit(X_train, Y_train) #Pasamos las variables de train, ya que nuestro test no tenemos disponible la Y

GridSearchCV(estimator=Pipeline(steps=[('std_slc', StandardScaler()),
                                       ('pca', PCA()), ('ridge', Ridge())]),
             param_grid={'pca__n_components': [1, 2, 3, 4, 5, 6],
                         'ridge__alpha': array([1.000e-03, 5.100e-02, 1.010e-01, 1.510e-01, 2.010e-01, 2.510e-01,
       3.010e-01, 3.510e-01, 4.010e-01, 4.510e-01, 5.010e-01, 5.510e-01,
       6.010e-01, 6.510e-01, 7.010e-01, 7.510e-01, 8.010e-01, 8.510e-01,...
       3.601e+00, 3.651e+00, 3.701e+00, 3.751e+00, 3.801e+00, 3.851e+00,
       3.901e+00, 3.951e+00, 4.001e+00, 4.051e+00, 4.101e+00, 4.151e+00,
       4.201e+00, 4.251e+00, 4.301e+00, 4.351e+00, 4.401e+00, 4.451e+00,
       4.501e+00, 4.551e+00, 4.601e+00, 4.651e+00, 4.701e+00, 4.751e+00,
       4.801e+00, 4.851e+00, 4.901e+00, 4.951e+00]),
                         'ridge__normalize': [True, False],
                         'ridge__solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                         

In [26]:
print("Best Number Of Components:", clf_GS.best_estimator_.get_params()["pca__n_components"])
print(); print(clf_GS.best_estimator_.get_params()["ridge"])

Best Number Of Components: 6

Ridge(alpha=3.851, solver='saga')


In [34]:
ridge = Ridge(alpha= 3.851, solver='saga').fit(X_train, Y_train)
print("Con los datos de entrenamiento: ", ridge.score(X_train, Y_train))

Con los datos de entrenamiento:  0.7486545086160235


##### Prueba 2 de encontrar el mejor alpha

In [28]:
ridge_2 = Ridge() #Creamos nuestro modelo

rango_alpha = {'alpha':np.arange(1e-3,5,0.05)}#Establecemos el rango con el que probar para nuestro alpha

ridge_regressor = GridSearchCV(ridge_2, rango_alpha, scoring="r2", cv = 10) #Creamos el GridSearch para que pruebe

ridge_regressor.fit(X_train,Y_train)

#Obtenemos los datos que son mejores
ridge_best_params_ = ridge_regressor.best_params_
ridge_best_score_ = -ridge_regressor.best_score_
print(ridge_best_params_); print(ridge_best_score_)

{'alpha': 0.901}
-0.7403519346584229


#### Conclusion de Ridge Regression

Aunque normalmente suele ser una buena opción usar algun derivado de Linear Regression mejor que el puro,aun usando dos técnicas para obtener la mejor combinación de hiperparametros, no he conseguido mejorar el score de la versión pura de LinearRegression.

    Aún así, dejo documentado las pruebas, por si fuese un error de ejecución o código en este caso.

### Polynomial signs

In [35]:
quad = PolynomialFeatures(degree = 2)
x_quad = quad.fit_transform(X_train)

X_train_quad,X_test_quad,Y_train_quad,Y_test_quad = train_test_split(x_quad,Y_train, random_state = 0)

plr = LinearRegression().fit(X_train_quad,Y_train_quad)

Y_train_pred = plr.predict(X_train_quad)
Y_test_pred = plr.predict(X_test_quad)

In [75]:
Y_train

0      11938.25595
1      12096.65120
2      14319.03100
3       3558.62025
4      13217.09450
          ...     
931    45008.95550
932    13470.80440
933    11286.53870
934     1880.48700
935    48675.51770
Name: charges, Length: 936, dtype: float64

In [74]:
df = pd.DataFrame(X_test_quad, columns = ['age','sex','bmi','children' ,'smoker', 'region'])
print(df)

ValueError: Shape of passed values is (234, 28), indices imply (234, 6)

In [38]:
print(plr.score(X_test_quad,Y_test_quad))

0.8724979280733158


## Modelado Deep Learning <a name="ModeladoDL"></a>

Elegiremos primero el tipo de DL:
 - **Red Neuronal Convolucional (CNN)**, este tipo de modelo esta diseñado exclusivamente para el reconocimiento de imagenes, por lo tanto lo descarto.
 - **Red Neuronal Recurrente (RNN)**, se utiliza principalmente para modelos que tengan como datos el lenguaje natural. Este tampoco es nuestro caso, por lo tanto lo descarto.
 - **AutoEncoder**, 
 - **Red Generativa Antagónica (GAN)**, 

In [94]:
insurance = Sequential([
      Dense(10),
      Dense(10),
      Dense(1),
])

insurance.compile(loss='mae',
                  optimizer='adam',
                  metrics=['mae'])

history = insurance.fit(X_train,
                        Y_train, validation_data=(X_train, Y_train), #No se muy bien que poner aqui
                        epochs=200, verbose=1)



Epoch 1/200
30/30 [==============================] - 0s 5ms/step - loss: 13542.5029 - mae: 13542.5029 - val_loss: 13541.8945 - val_mae: 13541.8945
Epoch 2/200
30/30 [==============================] - 0s 2ms/step - loss: 13541.2031 - mae: 13541.2031 - val_loss: 13540.2949 - val_mae: 13540.2949
Epoch 3/200
30/30 [==============================] - 0s 2ms/step - loss: 13539.2285 - mae: 13539.2285 - val_loss: 13537.8369 - val_mae: 13537.8369
Epoch 4/200
30/30 [==============================] - 0s 1ms/step - loss: 13536.2246 - mae: 13536.2246 - val_loss: 13534.1543 - val_mae: 13534.1543
Epoch 5/200
30/30 [==============================] - 0s 1ms/step - loss: 13531.7979 - mae: 13531.7979 - val_loss: 13528.7744 - val_mae: 13528.7744
Epoch 6/200
30/30 [==============================] - 0s 2ms/step - loss: 13525.4424 - mae: 13525.4424 - val_loss: 13521.2568 - val_mae: 13521.2568
Epoch 7/200
30/30 [==============================] - 0s 1ms/step - loss: 13516.7061 - mae: 13516.7061 - val_loss: 1351

Epoch 57/200
30/30 [==============================] - 0s 1ms/step - loss: 7972.1519 - mae: 7972.1519 - val_loss: 7936.9321 - val_mae: 7936.9321
Epoch 58/200
30/30 [==============================] - 0s 1ms/step - loss: 7911.8838 - mae: 7911.8838 - val_loss: 7883.6528 - val_mae: 7883.6528
Epoch 59/200
30/30 [==============================] - 0s 1ms/step - loss: 7859.2163 - mae: 7859.2163 - val_loss: 7835.3730 - val_mae: 7835.3730
Epoch 60/200
30/30 [==============================] - 0s 2ms/step - loss: 7812.3638 - mae: 7812.3638 - val_loss: 7790.2114 - val_mae: 7790.2114
Epoch 61/200
30/30 [==============================] - 0s 1ms/step - loss: 7769.8760 - mae: 7769.8760 - val_loss: 7746.7275 - val_mae: 7746.7275
Epoch 62/200
30/30 [==============================] - 0s 1ms/step - loss: 7730.5830 - mae: 7730.5830 - val_loss: 7710.6558 - val_mae: 7710.6558
Epoch 63/200
30/30 [==============================] - 0s 1ms/step - loss: 7695.2661 - mae: 7695.2661 - val_loss: 7676.2056 - val_mae: 76

Epoch 114/200
30/30 [==============================] - 0s 1ms/step - loss: 6683.0591 - mae: 6683.0591 - val_loss: 6670.8511 - val_mae: 6670.8511
Epoch 115/200
30/30 [==============================] - 0s 1ms/step - loss: 6660.7305 - mae: 6660.7305 - val_loss: 6648.4961 - val_mae: 6648.4961
Epoch 116/200
30/30 [==============================] - 0s 2ms/step - loss: 6638.0273 - mae: 6638.0273 - val_loss: 6625.3359 - val_mae: 6625.3359
Epoch 117/200
30/30 [==============================] - 0s 2ms/step - loss: 6615.0923 - mae: 6615.0923 - val_loss: 6602.9751 - val_mae: 6602.9751
Epoch 118/200
30/30 [==============================] - 0s 1ms/step - loss: 6592.5488 - mae: 6592.5488 - val_loss: 6579.3262 - val_mae: 6579.3262
Epoch 119/200
30/30 [==============================] - 0s 1ms/step - loss: 6568.6875 - mae: 6568.6875 - val_loss: 6556.0103 - val_mae: 6556.0103
Epoch 120/200
30/30 [==============================] - 0s 1ms/step - loss: 6544.7324 - mae: 6544.7324 - val_loss: 6531.8198 - val_

30/30 [==============================] - 0s 1ms/step - loss: 4787.5532 - mae: 4787.5532 - val_loss: 4764.2275 - val_mae: 4764.2275
Epoch 171/200
30/30 [==============================] - 0s 1ms/step - loss: 4744.5781 - mae: 4744.5781 - val_loss: 4720.6895 - val_mae: 4720.6895
Epoch 172/200
30/30 [==============================] - 0s 1ms/step - loss: 4700.9463 - mae: 4700.9463 - val_loss: 4677.2290 - val_mae: 4677.2290
Epoch 173/200
30/30 [==============================] - 0s 2ms/step - loss: 4656.5977 - mae: 4656.5977 - val_loss: 4632.6665 - val_mae: 4632.6665
Epoch 174/200
30/30 [==============================] - 0s 2ms/step - loss: 4612.1509 - mae: 4612.1509 - val_loss: 4588.1318 - val_mae: 4588.1318
Epoch 175/200
30/30 [==============================] - 0s 2ms/step - loss: 4567.1201 - mae: 4567.1201 - val_loss: 4542.8535 - val_mae: 4542.8535
Epoch 176/200
30/30 [==============================] - 0s 2ms/step - loss: 4521.5854 - mae: 4521.5854 - val_loss: 4495.6880 - val_mae: 4495.6880

## Evaluación <a name="Evaluacion"></a>

## Subir el modelo a Kaggle <a name="Subir"></a>

In [29]:
!pip install kaggle

In [61]:
X_train

,age,sex,bmi,children,smoker,region
0,0.869565,1.0,0.377184,0.0,0.0,0.000000
1,0.782609,0.0,0.337341,0.4,0.0,0.000000
2,1.000000,0.0,0.639871,0.0,0.0,1.000000
3,0.195652,0.0,0.233748,0.0,0.0,0.000000
4,0.913043,0.0,0.300154,0.0,0.0,0.000000
...,...,...,...,...,...,...
931,0.913043,0.0,0.437159,0.0,1.0,0.666667
932,0.956522,0.0,0.624773,0.0,0.0,0.666667
933,0.804348,0.0,0.363903,0.0,0.0,0.000000
934,0.043478,0.0,0.460925,0.0,0.0,1.000000


In [31]:
len(X_test.iloc[:,0:1].to_numpy().flatten())

402

In [ ]:
# Utilizamos el modelo que queramos mandar a Kaggle
predicted_charges = reg.predict(X_test.iloc[:,1:])
# Creamos el dataframe para mandarlo despues al CSV
submission = pd.DataFrame({'id': X_test.iloc[:,0:1].to_numpy().flatten(), 'charges': predicted_charges})
# Pasamos el dataframe a CSV
submission.to_csv('submission.csv', index=False)

Descomentar la siguiente línea para hacer la subida a kaggle a traves de su api, esto es posible porque tengo el token de mi perfil en mi ruta de la variable de entorno, por si quereis obtener más información visitar este link.
https://github.com/Kaggle/kaggle-api

In [33]:
#!kaggle competitions submit -c estimacin-de-costes-mdicos-sic-ed2-2021 -f submission.csv -m "Modelo ML con Regresion Lineal v2"

Pruebas de subir el otro modelo de Regression

In [64]:
print(X_test_quad)

[[1.         0.56521739 1.         ... 0.         0.         0.        ]
 [1.         0.45652174 0.         ... 0.         0.         1.        ]
 [1.         0.36956522 0.         ... 0.         0.         0.11111111]
 ...
 [1.         0.36956522 1.         ... 0.         0.         0.44444444]
 [1.         0.63043478 1.         ... 1.         1.         1.        ]
 [1.         0.39130435 0.         ... 0.         0.         0.44444444]]


In [54]:
len(Y_test_pred.flatten())

234

In [58]:
# Creamos el dataframe para mandarlo despues al CSV
submission = pd.DataFrame({'id': X_test_quad, 'charges': Y_test_pred})
# Pasamos el dataframe a CSV
submission.to_csv('submission.csv', index=False)

ValueError: Data must be 1-dimensional